In [1]:
import os
import scipy.io as io
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import sys
import math
from netCDF4 import Dataset
import os, calendar, sys, fnmatch, datetime

In [2]:
def extract_unique_values(input_array):
    unique_values = np.unique(input_array)
    return unique_values

# Load Huang et al. 2023 Ellipsoidal dust optical properties files

In [3]:
# ===== Aerosol (e.g., dust) optical properties (i.e., Qext, SSA, g) =====
# ***Note: The order of 14 bands is listed in TABLE 1 in RRTM_SW instruction (It is not monotonically increase or decrease)
# ***Note: The order of spectral optical properties should be consistent with the order in TABLE 1

def process_optical_properties(wavelength_band, txt_files):
    wavelength = wavelength_range[wavelength_band]
    #print(txt_files[wavelength_band])
    opt_prop_sw_original = np.loadtxt(opt_dir + '{}'.format(txt_files[wavelength_band]))
    #print(opt_prop_sw_original.shape)

    wl = np.array(opt_prop_sw_original[:,0]) #in um 
    dust_d = np.array(opt_prop_sw_original[:,1]) #in um
    size_parameter = np.pi * dust_d / wl #f'size_parameter_{wavelength:.4f}': size_parameter,
    n_RI = np.array(opt_prop_sw_original[:,2])
    k_RI = np.array(opt_prop_sw_original[:,3])
    
    qe_dust_sw = np.array(opt_prop_sw_original[:,4])
    mask = qe_dust_sw > 10
    qe_dust_sw[mask] = np.nan
    ssa_dust_sw = np.array(opt_prop_sw_original[:,5])
    ssa_dust_sw[mask] = np.nan
    g_dust_sw = np.array(opt_prop_sw_original[:,6])
    g_dust_sw[mask] = np.nan

    # Create variable names dynamically
    variables = {
        f'wl_{wavelength:.4f}': wl,
        f'dust_d_{wavelength:.4f}': dust_d,
        f'size_parameter_{wavelength:.4f}': size_parameter,
        f'n_RI_{wavelength:.4f}': n_RI,
        f'k_RI_{wavelength:.4f}': k_RI,
        f'qe_dust_sw_{wavelength:.4f}': qe_dust_sw,
        f'ssa_dust_sw_{wavelength:.4f}': ssa_dust_sw,
        f'g_dust_sw_{wavelength:.4f}': g_dust_sw,
    }
    return np.array((wl,dust_d,size_parameter,n_RI,k_RI,qe_dust_sw,ssa_dust_sw,g_dust_sw)) #variables


In [4]:
# SW Wavelength range and files
opt_dir = 'SW_Huang_Dust_Optics/'
wavelength_range = [0.35, 0.55, 0.75, 0.95] #, 2, 4, 6, 8, 10, 12, 14, 16, 35]
Huang_2023_dust_opt_prop_txt_files = ['shapein_TAMU_0.3500.txt', 'shapein_TAMU_0.5500.txt', 'shapein_TAMU_0.7500.txt', 'shapein_TAMU_0.9500.txt'] # 'shapein_TAMU_2.0000.txt', 'shapein_TAMU_4.0000.txt', 'shapein_TAMU_6.0000.txt', 'shapein_TAMU_8.0000.txt', 'shapein_TAMU_10.0000.txt', 'shapein_TAMU_12.0000.txt', 'shapein_TAMU_14.0000.txt', 'shapein_TAMU_16.0000.txt', 'shapein_TAMU_35.0000.txt']

# Array to store results
opt_prop_sw_array = np.empty(len(wavelength_range), dtype=object)

# Process optical properties for each wavelength band
for i in range(len(wavelength_range)):
    opt_prop_sw = process_optical_properties(i, Huang_2023_dust_opt_prop_txt_files)
    opt_prop_sw_array[i] = opt_prop_sw

In [5]:
opt_prop_sw_array[0][2].shape
unique_size_param = np.empty(len(wavelength_range), dtype=object)
for i in range(len(wavelength_range)):
    unique_size_param[i] = extract_unique_values(opt_prop_sw_array[i][2])
print(unique_size_param.shape) #300 for each wavelength 
#choose every 10 for each wavelength 

#then chose every 10, and sort them (it's okay that it is logarithmic)

(4,)


## Create Yue Size parameter, real RI, and imaginary RI matrix

create a file of the three dimensional grid, those three dimensions being size parameter, real RI, and imaginary RI. The second and third dimensions here are easier and would be those values relevant to your problem, so I'd suggest (1.50, 1.52, 1.53, 1.56) and (0.0005, 0.001, 0.002, 0.003). The first dimension is pretty annoying because there will be so many values since the data is finely spaced in D (300 values per wavelength) and you will have four different SW wavelengths to use (350, 550, 750, and 950 nm; don't use the other ones because the RI there is for the LW). So I'd suggest only using every 10th value to keep the size of the matrix reasonable (120 size parameter values). Once you've created this three-dimensional grid (120x4x4), 

In [25]:
Yue_real_RI_range = np.array([1.50, 1.52, 1.53, 1.56])
Yue_imag_RI_range = np.array([0.0005, 0.001, 0.002, 0.003])
Yue_size_parameter_range = [] #np.zeros(120, dtype=object)
for i in range(0,len(wavelength_range)):
    for j in range(0,len(unique_size_param[i]),10):
        Yue_size_parameter_range.append(unique_size_param[i][j])
Yue_size_parameter_range = np.sort(Yue_size_parameter_range)
Yue_size_parameter_range = np.array(Yue_size_parameter_range)
#print(Yue_size_parameter_range)

#Yue_size_parameter_and_Ri_matrix = (Yue_size_parameter_range, Yue_real_RI_range, Yue_imag_RI_range)

Yue_size_parameter_and_Ri_matrix = np.array((Yue_size_parameter_range, Yue_real_RI_range, Yue_imag_RI_range),dtype=object)
io.savemat("coordinate_matrix_tuple.mat", {"array": Yue_size_parameter_and_Ri_matrix})
print(Yue_size_parameter_and_Ri_matrix)

[array([3.30693964e-01, 4.16674394e-01, 4.18879020e-01, 5.25142014e-01,
        5.27787566e-01, 5.71198664e-01, 6.62049315e-01, 6.65179885e-01,
        7.19710317e-01, 8.34340870e-01, 8.38595799e-01, 8.97597901e-01,
        9.07063479e-01, 1.05160680e+00, 1.05683177e+00, 1.13097336e+00,
        1.14353973e+00, 1.32542141e+00, 1.33203529e+00, 1.42538547e+00,
        1.44113423e+00, 1.67033521e+00, 1.67886711e+00, 1.79699100e+00,
        1.81641175e+00, 2.10519777e+00, 2.11575793e+00, 2.26463950e+00,
        2.28936425e+00, 2.65315767e+00, 2.66658384e+00, 2.85436133e+00,
        2.88512445e+00, 3.34397736e+00, 3.36066638e+00, 3.59757239e+00,
        3.63625070e+00, 4.21436387e+00, 4.23570466e+00, 4.53376700e+00,
        4.58272688e+00, 5.31127575e+00, 5.33819424e+00, 5.71410824e+00,
        5.77596089e+00, 6.69390721e+00, 6.72761595e+00, 7.20142796e+00,
        7.27935578e+00, 8.43666440e+00, 8.47894913e+00, 9.07650998e+00,
        9.17402175e+00, 1.06328030e+01, 1.06864416e+01, 1.143898

In [24]:
print(type(Yue_size_parameter_and_Ri_matrix))

<class 'tuple'>


Yue_coord = np.zeros((len(Yue_size_parameter_range),len(Yue_real_RI_range),len(Yue_imag_RI_range)), dtype=object)
for i in range(len(Yue_size_parameter_range)):
    for j in range(len(Yue_real_RI_range)):
        for k in range(len(Yue_imag_RI_range)):
            Yue_coord[i,j,k] = (Yue_size_parameter_range[i],Yue_real_RI_range[j],Yue_imag_RI_range[k])
print(Yue_coord.shape, Yue_coord)
io.savemat("Yue_coord.mat", {"array": Yue_coord})

## Create Yue Q_e matrix

 you need to create a matrix with the same dimensions as your three-dimensional grid that contains the values of Q_ext. You do this by creating a matrix with NaN values with those dimensions and then reading in all Yue's data (at the four wavelengths, using only every tenth data point) and writing each value to this new matrix at the corresponding coordinate. Once you're done, you'll end with a matrix that has a lot of values filled in, but ~75% of the values will be NaNs. 

In [21]:
len(opt_prop_sw_array[0][5])

14400

In [22]:
#print(coordinate_matrix)

In [23]:
def populate_Qe_matrix(Qe_matrix, wavelength_range, Yue_size_parameter_and_Ri_matrix, opt_prop_sw_array):
    """
    Populate Qe_matrix based on conditions for all wavelengths.

    Parameters:
    - Qe_matrix: numpy array
    - wavelength_range: range
    - Yue_size_parameter_and_Ri_matrix: list
    - opt_prop_sw_array: list

    Returns:
    - Qe_matrix: numpy array, populated Qe_matrix
    """
    #Qe_tuple =[]
    for iwl in wavelength_range:
        for i in range(len(Yue_size_parameter_and_Ri_matrix[0])):
            for j in range(len(Yue_size_parameter_and_Ri_matrix[1])):
                for k in range(len(Yue_size_parameter_and_Ri_matrix[2])):
                    if np.isnan(Qe_matrix[i, j, k]):
                        for w in range(0, len(opt_prop_sw_array[iwl][5]), 10):
                            if (Yue_size_parameter_and_Ri_matrix[0][i] == opt_prop_sw_array[iwl][2][w] and
                                Yue_size_parameter_and_Ri_matrix[1][j] == opt_prop_sw_array[iwl][3][w] and 
                                Yue_size_parameter_and_Ri_matrix[2][k] == opt_prop_sw_array[iwl][4][w]):
                                    Qe_matrix[i, j, k] = opt_prop_sw_array[iwl][5][w]
                                    #Qe_tuple.append(opt_prop_sw_array[iwl][5][w])
    return Qe_matrix

# Example data (replace with your actual data)
wavelength_range = range(len(wavelength_range))

# Set up Qe_matrix with nan values
Qe_matrix = np.full((120, 4, 4), np.nan, dtype=float)

# Populate Qe_matrix based on conditions for all wavelengths
Qe_matrix = populate_Qe_matrix(Qe_matrix, wavelength_range, Yue_size_parameter_and_Ri_matrix, opt_prop_sw_array)
io.savemat("Qe_matrix.mat", {"array":Qe_matrix})

In [15]:
def count_nan_values(matrix):
    """
    Count the number of nan values in a multidimensional array.

    Parameters:
    - matrix: numpy array

    Returns:
    - count: int, number of nan values
    """
    count = np.sum(np.isnan(matrix))
    return count

nan_count = count_nan_values(Qe_matrix )
print("Number of nan values:", nan_count)

Number of nan values: 80


In [16]:
def populate_SSA_matrix(SSA_matrix, wavelength_range, Yue_size_parameter_and_Ri_matrix, opt_prop_sw_array):
    """
    Populate SSA_matrix based on conditions for all wavelengths.

    Parameters:
    - SSA_matrix: numpy array
    - wavelength_range: range
    - Yue_size_parameter_and_Ri_matrix: list
    - opt_prop_sw_array: list

    Returns:
    - SSA_matrix: numpy array, populated SSA_matrix
    """
    #SSA_tuple =[]
    for iwl in wavelength_range:
        for i in range(len(Yue_size_parameter_and_Ri_matrix[0])):
            for j in range(len(Yue_size_parameter_and_Ri_matrix[1])):
                for k in range(len(Yue_size_parameter_and_Ri_matrix[2])):
                    if np.isnan(SSA_matrix[i, j, k]):
                        for w in range(0, len(opt_prop_sw_array[iwl][5]), 10):
                            if (Yue_size_parameter_and_Ri_matrix[0][i] == opt_prop_sw_array[iwl][2][w] and
                                Yue_size_parameter_and_Ri_matrix[1][j] == opt_prop_sw_array[iwl][3][w] and 
                                Yue_size_parameter_and_Ri_matrix[2][k] == opt_prop_sw_array[iwl][4][w]):
                                    SSA_matrix[i, j, k] = opt_prop_sw_array[iwl][6][w]
                                    #SSA_tuple.append(opt_prop_sw_array[iwl][6][w])
    return SSA_matrix

# Example data (replace with your actual data)
wavelength_range = range(len(wavelength_range))

# Set up SSA_matrix with nan values
SSA_matrix = np.full((120, 4, 4), np.nan, dtype=float)

# Populate SSA_matrix based on conditions for all wavelengths
SSA_matrix = populate_SSA_matrix(SSA_matrix, wavelength_range, Yue_size_parameter_and_Ri_matrix, opt_prop_sw_array)
io.savemat("SSA_matrix.mat", {"array":SSA_matrix})

In [17]:
nan_count = count_nan_values(SSA_matrix )
print("Number of nan values:", nan_count)

Number of nan values: 80


In [18]:
def populate_g_matrix(g_matrix, wavelength_range, Yue_size_parameter_and_Ri_matrix, opt_prop_sw_array):
    """
    Populate g_matrix based on conditions for all wavelengths.

    Parameters:
    - g_matrix: numpy array
    - wavelength_range: range
    - Yue_size_parameter_and_Ri_matrix: list
    - opt_prop_sw_array: list

    Returns:
    - g_matrix: numpy array, populated g_matrix
    """
    #g_tuple = []
    for iwl in wavelength_range:
        for i in range(len(Yue_size_parameter_and_Ri_matrix[0])):
            for j in range(len(Yue_size_parameter_and_Ri_matrix[1])):
                for k in range(len(Yue_size_parameter_and_Ri_matrix[2])):
                    if np.isnan(g_matrix[i, j, k]):
                        for w in range(0, len(opt_prop_sw_array[iwl][5]), 10):
                            if (Yue_size_parameter_and_Ri_matrix[0][i] == opt_prop_sw_array[iwl][2][w] and
                                Yue_size_parameter_and_Ri_matrix[1][j] == opt_prop_sw_array[iwl][3][w] and 
                                Yue_size_parameter_and_Ri_matrix[2][k] == opt_prop_sw_array[iwl][4][w]):
                                    g_matrix[i, j, k] = opt_prop_sw_array[iwl][7][w]
                                    #g_tuple.append(opt_prop_sw_array[iwl][7][w])
    return g_matrix

# Example data (replace with your actual data)
wavelength_range = range(len(wavelength_range))

# Set up SSA_matrix with nan values
g_matrix = np.full((120, 4, 4), np.nan, dtype=float)

# Populate g_matrix based on conditions for all wavelengths
g_matrix = populate_g_matrix(g_matrix, wavelength_range, Yue_size_parameter_and_Ri_matrix, opt_prop_sw_array)
io.savemat("g_matrix.mat", {"array":g_matrix})

In [19]:
nan_count = count_nan_values(g_matrix )
print("Number of nan values:", nan_count)

Number of nan values: 80


# Calculate DustCOMM mean geometric diamater of each bin

In [ ]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

daod_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_DAOD_annual_bin_abs.nc')
print(daod_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

lat_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lat'][:])
lon_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_Daod_Dustcomm_20PM = np.array(daod_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
dustcomm_Sources_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['source'][:]) 
dustcomm_bin_lower = np.array(daod_Dustcomm_PM20_Dataset.variables['bin_D_lower'][:])
dustcomm_bin_upper = np.array(daod_Dustcomm_PM20_Dataset.variables['bin_D_upper'][:])

In [ ]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

loading_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_dust_loading_annual_bin_abs.nc')
print(loading_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

loading_lat_Dustcomm_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['lat'][:])
loading_lon_Dustcomm_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_loading_Dustcomm_20PM = np.array(loading_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
loading_dustcomm_Sources_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['source'][:]) 
loading_dustcomm_bin_lower = np.array(loading_Dustcomm_PM20_Dataset.variables['bin_D_lower'][:])
loading_dustcomm_bin_upper = np.array(loading_Dustcomm_PM20_Dataset.variables['bin_D_upper'][:])

In [ ]:
print(dustcomm_bin_lower)
print(dustcomm_bin_upper)

In [ ]:
mean_geometric_diameter_bin = np.sqrt(dustcomm_bin_lower*dustcomm_bin_upper)

In [ ]:
print(mean_geometric_diameter_bin)

# Interpolate  Huang et al. 2023 size parameter and dust optical properties that best match Adebiyi RI's, and DustCOMM size parameter of each bin

use DustCOMM Saharan and Sahelian dust partition in a grid box, in the DAOD. sum the two sahara sources, to calculate my own spatially resolved RI (Later)

match_d_opt_prop_sw = []
match_n_RI_opt_prop_sw = []
match_k_RI_opt_prop_sw = []
n_RI_index = []
k_RI_index = []North African dust has a mean imaginary refractive index at 550 nm wavelength of 0.0012 (one standard error range of 0.0009–0.0016
and a real dust refractive index, nr = 1.51 ± 0.03 (which is the same for Sahara and Sahel) (Di Biagio et al. 2019)
We assume that RI is constant across particle size (Adebiyi et al. 2023, Obiso et al. 2023)

In [ ]:
# 14 RRTM SW bands - average wavelength of each band (in RRTM order)
RRTM_SW_bands = [3.4615, 2.7885, 2.3255, 2.0465, 1.784, 1.4625, 1.2705, 1.01, 0.7015, 0.5335, 0.3935, 0.304, 0.2315, 8.0205]
RRTM_SW_bands_sorted = np.sort(RRTM_SW_bands)

In [ ]:
#grid_box = [0,18] #0W, 18N
North_African_dust_adebiyi_k = 0.0012
North_African_dust_adebiyi_nr = 1.51 

Meeting Notes 1/4/24:

Look at how solar zenith angle variation effects "DRE" (difference between the two fluxes)
Send Jasper bulk dust optical properties by end of day tomorrow (plus text file)
write script to run SW RRTM for different solar zenith angles

## 1/22/24 Interpolate for each dust optical property

In [ ]:
#from scipy.interpolate import RegularGridInterpolator
#from scipy.interpolate import griddata 
import scipy

In [ ]:
Yue_coordinate_matrix = Yue_size_parameter_and_Ri_matrix
print(Yue_coord_matrix.shape)
print(Qe_matrix.shape)
print(SSA_matrix.shape)
print(g_matrix.shape)

In [ ]:
Qe_interp_bin_wavelength_array = np.zeros((len(mean_geometric_diameter_bin),len(RRTM_SW_bands)), dtype=object)
for ibin in range(len(mean_geometric_diameter_bin)):
    for irrtm in range(len(RRTM_SW_bands)):
        size_param_RRTM = np.pi*mean_geometric_diameter_bin[ibin]/RRTM_SW_bands_sorted[irrtm] #not sorted, in RRTM order
        point =  (size_param_RRTM, North_African_dust_adebiyi_nr, North_African_dust_adebiyi_k)
        Qe_interp = scipy.interpolate.interpn(Yue_coord, Qe_matrix, point , method='linear')
        #Qe_interp = scipy.interpolate.interpn(Yue_coord_matrix, Qe_matrix, point , method='linear')
        #Qe_interp = scipy.interpolate.interpn(Yue_size_parameter_and_Ri_matrix,Qe_matrix, point , method='linear')
        Qe_interp_bin_wavelength_array[ibin,irrtm] =  Qe_interp                                    

# Calculate bulk dust Optical properties for (0W, 18N)

Reorder dust optical properties so it is in order (Q_e, ssa, g)

In [ ]:
# Calculate and print for each bin
all_bins_opt_prop_array = []

# Convert the list to a NumPy array
all_bins_opt_prop_array = np.array((bin1_opt_prop_interp_size_param_y, bin2_opt_prop_interp_size_param_y, bin3_opt_prop_interp_size_param_y, bin4_opt_prop_interp_size_param_y, bin5_opt_prop_interp_size_param_y, bin6_opt_prop_interp_size_param_y))
print(all_bins_opt_prop_array)

In [ ]:
print(loading_lat_Dustcomm_PM20.shape)
print(loading_lon_Dustcomm_PM20.shape)
print(mean_loading_Dustcomm_20PM.shape)
print(mean_loading_Dustcomm_20PM[0,0,1,2])
#dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
index_0W = []
index_18N = []

lat_0W = 0
lon_18N = 18

for ilat in range(0,len(loading_lat_Dustcomm_PM20)):
    if np.round(loading_lat_Dustcomm_PM20[ilat]) -1 == lat_0W:
        print(loading_lat_Dustcomm_PM20[ilat])
        index_0W.append(ilat)

for ilon in range(0,len(loading_lon_Dustcomm_PM20)):
    if np.round(loading_lon_Dustcomm_PM20[ilon]) == lon_18N: 
        print(loading_lon_Dustcomm_PM20[ilon])
        index_18N.append(ilon)
print(index_0W,index_18N)

In [ ]:
total_North_African_loading = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1] + mean_loading_Dustcomm_20PM[:,:,:,2]
Sarahan_laoding = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1]
Sahelian_loading = mean_loading_Dustcomm_20PM[:,:,:,2]
print(total_North_African_loading.shape)

In [ ]:
#ρ_d=(2.5±0.2)×10^3 kg m-3 is the globally representative density of dust aerosols (Fratini et al., 2007; Reid et al., 2008; Kaaden et al., 2009; Sow et al., 2009)
ρ_d = 2.5e3
delta_m_Db = total_North_African_loading[48,78,:]
print('delta_m_Db',delta_m_Db)

Q_e_bulk_0W_18N_bin_num = np.zeros(6)
Q_e_bulk_0W_18N_bin_den = np.zeros(6)
SSA_bulk_0W_18N_bin_num = np.zeros(6)
SSA_bulk_0W_18N_bin_den = np.zeros(6)
Q_scat = np.zeros(6)
g_bulk_0W_18N_bin_num = np.zeros(6)
g_bulk_0W_18N_bin_den = np.zeros(6)

#all_bins_opt_prop_array[i][0] = Q_e of bin i
#all_bins_opt_prop_array[i][1] = SSA of bin i
#all_bins_opt_prop_array[i][2] = g of bin i

for i in range(0,len(all_bins_opt_prop_array)):
    Q_e_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    Q_e_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2))/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    
    SSA_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0]*all_bins_opt_prop_array[i][1])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    SSA_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    
    Q_scat[i] = all_bins_opt_prop_array[i][1] * all_bins_opt_prop_array[i][0]
    
    g_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*Q_scat[i]*all_bins_opt_prop_array[i][2])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    g_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*Q_scat[i])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    

In [ ]:
# Calculate values
Q_e_bulk_0W_18N_bulk = np.sum(Q_e_bulk_0W_18N_bin_num) / np.sum(Q_e_bulk_0W_18N_bin_den)
SSA_bulk_0W_18N_bulk = np.sum(SSA_bulk_0W_18N_bin_num) / np.sum(SSA_bulk_0W_18N_bin_den)
g_bulk_0W_18N_bulk = np.sum(g_bulk_0W_18N_bin_num) / np.sum(g_bulk_0W_18N_bin_den)

# Print the values
print("Q_e_bulk_0W_18N_bulk: ", Q_e_bulk_0W_18N_bulk)
print("SSA_bulk_0W_18N_bulk: ", SSA_bulk_0W_18N_bulk)
print("g_bulk_0W_18N_bulk: ", g_bulk_0W_18N_bulk)

In [ ]:
def create_if_not_exists(file_path):
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            # You can optionally write initial content to the file
            file.write("Wavelength\tQ_e_bulk_0W_18N_bulk\tSSA_bulk_0W_18N_bulk\tg_bulk_0W_18N_bulk\t" + '\n')
            #file.write(f"Q_e_bulk_0W_18N_bulk\t")
            #file.write(f"SSA_bulk_0W_18N_bulk\t")
            #file.write(f"g_bulk_0W_18N_bulk\t\n")
        print(f"File '{file_path}' created.")
    else:
        print(f"File '{file_path}' already exists.")

# Check if dust optical properties txt file already exists
file_path1 = 'RRTM_bulk_dust_optical_properties.txt'
file_path2 = 'RRTM_bulk_dust_optical_properties_{}.txt'.format(wavelength)
create_if_not_exists(file_path1)
create_if_not_exists(file_path2)

In [ ]:
# Assuming Q_e_bulk_0W_18N_bulk, SSA_bulk_0W_18N_bulk, g_bulk_0W_18N_bulk are numpy arrays

# Data to append
data_to_append = np.column_stack((wavelength, Q_e_bulk_0W_18N_bulk, SSA_bulk_0W_18N_bulk, g_bulk_0W_18N_bulk))

# Open the file in append mode and use np.savetxt to append data
with open('bulk_dust_optical_properties_{}.txt'.format(wavelength), 'a') as file:
    np.savetxt(file, data_to_append, fmt='%5.3f', delimiter='\t', header='', comments='')
    
# Open the file in append mode and use np.savetxt to append data
with open('bulk_dust_optical_properties.txt', 'a') as file:
    np.savetxt(file, data_to_append, fmt='%5.3f', delimiter='\t', header='', comments='')